A diferencia de los modelos anteriores que consolidaba datos por período de consumos, en el presente script se busca pegar el consumo por PERIODO PRESTACION por persona y cada rango separado. 
Se eliminan cálculos importantes como la edad de la persona al momento del conusmo.

In [1]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
# Parámetros de seteo.

# Parámetros de seteo.

# Ruta a la que tiene que salir la información
ruta_base = 'C:\\Users\\lsaucedo\\Desktop\\Brinkmann v2\\'

# Archivo de salida
archivo_salida = ruta_base + "PerPrest-Persona-Consumo-Bnknn_v2" + ".csv"

# Ruta en que se encuentran los archivos:
ruta = ruta_base + 'Consumo_Zona\\'

ruta_aux = 'C:\\modelos\\Auxiliares\\'

# 1- Levantar los archivos con consumos

In [3]:
# Levanto el consumo existente en una carpeta:

# Buscamos y levantamos a df todos los excel de "ruta"
contenido = os.listdir(ruta)

print("Inicio del proceso: " + datetime.now().strftime('%H:%M:%S'))

# Dataframe con el stock:
dfc = pd.DataFrame()

archivos = []
for fichero in contenido:
    if os.path.isfile(os.path.join(ruta, fichero)) and fichero.endswith('.xlsx'):
        df_temp = pd.read_excel(ruta+fichero)
        dfc = pd.concat([dfc, df_temp], axis=0)
        print("Se cargó el archivo: ", fichero, "a las: " + datetime.now().strftime('%H:%M:%S'))

df_temp = []
print("Fin de la carga de archivos")

Inicio del proceso: 17:32:32
Se cargó el archivo:  Cons_Brinkmann_202010_202109.xlsx a las: 17:34:08
Fin de la carga de archivos


In [4]:
dfcbkp = dfc
#dfc = dfcbkp 

# 1.2- Renombrar las columnas

In [5]:
# Se pueden renombrar incluso columnas que no existen (mejorar la eficiencia en función del warning)
dfc.rename( columns = {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE', 
                           'Consumos Costo con Subsidios SSS': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3',
                           'Precio Sugerido Medicamento': "Precio_sug",
                           'Periodo Prestacion ID':'Periodo_Prest',
                           'Provision Acreedor DESC':"Prov_acr_de",
                           'Porcentaje Cobertura Medicamento ID': "Cobertura_DW",
                           'Prestacion ID': 'Prestacion',
                           'Consumos Costo con Otros Subsidios': 'Consumo',
                           'Cantidad Practicas Pagadas': 'Cantidad',
                           'Plan Agrupado Con Segmento DESC':'Plan Agrupado'
                           }, inplace=True)

# 2- Cálculo de edad

No se realiza el cálculo a partir de la fecha de nacimiento y período. Par eso ver versines 0.x

# 2- Agrego archivos auxiliares

No se agregan los diagnósticos

# 3- Agrego Marcas de Consumo y Rango Consumos

No se agregan rangos de consumos en esta versión. Bucar en las v0.x.

# Inspecciones varias

In [6]:
dfc['Nomenclador ID'].unique()

array(['DI', 'NM', 'ME', 'MD', 'ND', 'NU', 'NO', 'OP', 'NB', 'NP', 'M1',
       'HB', nan, 'NC', 'NS'], dtype=object)

In [7]:
dfc['Origen Prestacion ID'].unique()

array(['A', 'I', 'X'], dtype=object)

In [8]:
dfc['Origen Facturacion ID'].unique()

array(['F', 'R', 'A', 'C'], dtype=object)

# Agregando Marca PMO SanCor

In [9]:
# Levanto el excel con marca de prestaciones pmo
dfpmo = pd.read_excel(ruta_aux + 'Aux_pmo.xlsx')

In [10]:
# Quito los espacios a las prestaciones:
dfc["Prest_sin_esp"] = dfc.Prestacion.str.strip()

In [11]:
dfc = pd.merge( left = dfc, right = dfpmo, left_on='Prest_sin_esp', 
               right_on='Prest sin espacios', how = "left")

# Libero memoria:
dfpmo = []

In [12]:
# Prestaciones detectadas como faltantes:
dfc[dfc['Marca PMO'].isna()]["Prestacion"].unique()

array(['NS121002  ', 'NS0405710 ', 'NS0401070 ', 'NS0401100 ',
       'NS0401080 ', 'NS0401140 '], dtype=object)

In [13]:
# Reemplazamos los nan de marca pmo por "NO PMO"
dfc['Marca PMO'] = dfc['Marca PMO'].replace(np.nan,"NO PMO")

# Segmentaciones y métricas

In [14]:
# Segmentaciones y métricas comúnes:

# QUEDA PENDIENTE SEGMENTO EN LA APERTURA
segmentacion = (['Periodo_Prest', 'Persona'])

# dejo para probar procesamientos más rápido (debugging)
segmentacion2 = ['Periodo', 'Sexo']


# Defino las métricas a calcular
metricas = ({'Consumo':[np.sum], 'Prestacion':pd.Series.nunique,'Cantidad':[np.sum]})


metricas2 = ({  'Consumo':[np.sum,len],
                'Cantidad':np.sum,
            })


# TD1 - Armado de la primera tabla de resúmen:
## Período PRESTACION - Persona
## A1 = Total
Se calcula para diferentes aperturas.

In [15]:
# Genero la tabla de salida
td1 = pd.pivot_table(dfc, index = segmentacion, 
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas)

In [16]:
# Renombramos columnas:

# Reseteo los multi index:
# td0 = td0.reset_index()


# Nombres de columnas:
col_td1 = [

    "Cantidad Total",
    "Consumo Total",
    "Prest. Dist. Total"  ]


# Renombro las columnas:
td1.columns = col_td1

# TD2 - Armado de aperturas diferentes al total
## A2: Por nomenclador y Origen Prestación

In [17]:
# Apertura para consumo agrupacion 2 (la 1 es la total)

condiciones2 = [
    (dfc['Origen Facturacion ID'] == 'A') ,
    (dfc['Origen Facturacion ID'] == 'C') ,
    (dfc['Nomenclador ID'] == 'ME') ,
    (dfc['Nomenclador ID'] == 'M1') ,
    (dfc['Origen Prestacion ID'] == 'I')]

# Lista de resultados en función de las selecciones
valores_condic2 = ['Naj', 'NC', 'MEd', 'MEd', 'Internacion sin A,C,Med']     

# Generación de la columna calculada
dfc['Agrc2'] = np.select(condiciones2, valores_condic2, default = 'Ambulatorio sin A,C,Med')

In [18]:
# No debería haber nan
dfc['Agrc2'].unique()

array(['Ambulatorio sin A,C,Med', 'Internacion sin A,C,Med', 'MEd', 'Naj',
       'NC'], dtype=object)

In [19]:
td2 = pd.pivot_table(dfc, index = segmentacion, columns = 'Agrc2',
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas, fill_value = 0)

In [20]:
# Nombres de columnas:
col_td2 = [
    
    "Cantidad Amb.",
    "Cantidad Int.",
    "Cantidad Medic.",
    "Cantidad NC.",
    "Cantidad NA",
    
    "Consumo Amb.",
    "Consumo Int.",
    "Consumo Medic.",
    "Consumo NC.",
    "Consumo NA",
    
    "Prest. Dist. Amb.",
    "Prest. Dist. Int.",
    "Prest. Dist. Medic.",
    "Prest. Dist. NC.",
    "Prest. Dist. NA"]

# Renombro las columnas:
td2.columns = col_td2


# TD2 - Amb por Rubros

## A3

In [21]:
# Apertura para consumo: Agrupador consumos 3

condiciones3 = [
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'A'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'I'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'C'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'D'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'E'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'F'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'G'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'N'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'M'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'J'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'R'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'S'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'T'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'U'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'V'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'W'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'X'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'M1'),

    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'NP'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'DI'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'NC'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'NA'),
    
    (dfc['Origen Prestacion ID'] == 'A') ]

# Lista de resultados en función de las selecciones
valores_condic3 = ['Consulta', 'Laboratorio', 'Imagenes', 'Imagenes', 'Imagenes', 
                  'Imagenes', 'Imagenes', 'Fisiokinesio', 'Salud Mental', 
                  'Odontologia', 'Odontologia', 'Odontologia', 'Odontologia', 
                   'Odontologia', 'Odontologia', 'Odontologia', 'Odontologia',
                   'Amb ME','Amb ME','Amb NP','Amb DI', 'Amb NC',
                   'Amb NA','Resto Amb']   

# Generación de la columna calculada
dfc['Agrc3'] = np.select(condiciones3, valores_condic3, default = 'No Considerar')


In [22]:
# Armado de la td:

td3 = pd.pivot_table(dfc, index = segmentacion, columns = 'Agrc3',
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas, fill_value = 0)

# td3.head()

In [23]:
td3.columns

MultiIndex([(  'Cantidad',     'sum',        'Amb DI'),
            (  'Cantidad',     'sum',        'Amb ME'),
            (  'Cantidad',     'sum',        'Amb NC'),
            (  'Cantidad',     'sum',        'Amb NP'),
            (  'Cantidad',     'sum',      'Consulta'),
            (  'Cantidad',     'sum',  'Fisiokinesio'),
            (  'Cantidad',     'sum',      'Imagenes'),
            (  'Cantidad',     'sum',   'Laboratorio'),
            (  'Cantidad',     'sum', 'No Considerar'),
            (  'Cantidad',     'sum',   'Odontologia'),
            (  'Cantidad',     'sum',     'Resto Amb'),
            (  'Cantidad',     'sum',  'Salud Mental'),
            (   'Consumo',     'sum',        'Amb DI'),
            (   'Consumo',     'sum',        'Amb ME'),
            (   'Consumo',     'sum',        'Amb NC'),
            (   'Consumo',     'sum',        'Amb NP'),
            (   'Consumo',     'sum',      'Consulta'),
            (   'Consumo',     'sum',  'Fisiokin

In [24]:
# Nombres de columnas:
col_td3 = [
    "Cantidad Amb DI",    
    "Cantidad Amb ME",
    "Cantidad Amb NC",
    "Cantidad Amb NP",
    "Cantidad Consulta",
    "Cantidad Fisiokinesio",
    "Cantidad Imagenes",
    "Cantidad Laboratorio",
    "Cantidad No Considerar",
    "Cantidad Odontologia",
    "Cantidad Resto Amb",
    "Cantidad Salud Mental",
    
    
    "Consumo Amb DI",    
    "Consumo Amb ME",
    "Consumo Amb NC",
    "Consumo Amb NP",
    "Consumo Consulta",
    "Consumo Fisiokinesio",
    "Consumo Imagenes",    
    "Consumo Laboratorio",
    "Consumo No Considerar",
    "Consumo Odontologia",
    "Consumo Resto Amb",
    "Consumo Salud Mental",

    "Prest. Dist. Amb DI",    
    "Prest. Dist. Amb ME",
    "Prest. Dist. Amb NC",
    "Prest. Dist. Amb NP",    
    "Prest. Dist. Consulta",
    "Prest. Dist. Fisiokinesio",
    "Prest. Dist. Imagenes",    
    "Prest. Dist. Laboratorio",
    "Prest. Dist. No Considerar",
    "Prest. Dist. Odontologia",
    "Prest. Dist. Resto Amb",
    "Prest. Dist. Salud Mental"]

# Renombro las columnas:
td3.columns = col_td3

# TD4 - Medicamentos por Origen Prestación

## Agr 4: Medicamentos por origen

In [25]:
# Apertura para consumo: Agrupador consumos 3

condiciones4 = [
    
    # Condiciones para marcar medicamentos en internacion:
    (dfc['Origen Prestacion ID'] == 'I') & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Origen Prestacion ID'] == 'I') & (dfc['Nomenclador ID'] == 'M1'),
    
    # Condiciones para marcar medicamentos en amb -> no se filtra origen (por defecto mandar a AMB)
    (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Nomenclador ID'] == 'M1') ]

# Lista de resultados en función de las selecciones
valores_condic4 = ['Med. Int', 'Med. Int', 'Med. Amb', 'Med. Amb']     

# Generación de la columna calculada
dfc['Agrc4'] = np.select(condiciones4, valores_condic4, default = 'No Considerar')


In [26]:
# Armado de la td:

td4 = pd.pivot_table(dfc, index = segmentacion, columns = 'Agrc4',
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas, fill_value = 0)

# td4.head()

In [27]:
td4.columns

MultiIndex([(  'Cantidad',     'sum',      'Med. Amb'),
            (  'Cantidad',     'sum',      'Med. Int'),
            (  'Cantidad',     'sum', 'No Considerar'),
            (   'Consumo',     'sum',      'Med. Amb'),
            (   'Consumo',     'sum',      'Med. Int'),
            (   'Consumo',     'sum', 'No Considerar'),
            ('Prestacion', 'nunique',      'Med. Amb'),
            ('Prestacion', 'nunique',      'Med. Int'),
            ('Prestacion', 'nunique', 'No Considerar')],
           names=[None, None, 'Agrc4'])

In [28]:
# Nombres de columnas:
col_td4 = [
    
    "Cantidad Med. Amb",
    "Cantidad Med. Int",
    "Cantidad No Considerar",
    
    "Consumo Med. Amb",
    "Consumo Med. Int",
    "Consumo No Considerar",    


    "Prest. Dist. Med. Amb",
    "Prest. Dist. Med. Int",
    "Prest. No Considerar"]

# Renombro las columnas:
td4.columns = col_td4

# TD5 - Medicamentos por vía

## Agr 5: Medicamentos por vía

In [29]:
# Apertura para consumo: Agrupador consumos 3

condiciones5 = [
    
    # Condiciones para marcar medicamentos en internacion:
    (dfc['Provision Acreedor ID'] == 6) & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Provision Acreedor ID'] == 6) & (dfc['Nomenclador ID'] == 'M1'),

    (dfc['Provision Acreedor ID'] == 66) & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Provision Acreedor ID'] == 66) & (dfc['Nomenclador ID'] == 'M1'),
    
    (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Nomenclador ID'] == 'M1') ]

# Lista de resultados en función de las selecciones
valores_condic5 = ['Farmacia', 'Farmacia', 'Provision', 'Provision', 'Otras vias', 'Otras vias']     

# Generación de la columna calculada
dfc['Agrc5'] = np.select(condiciones5, valores_condic5, default = 'No Considerar')


In [30]:
# Armado de la td:

td5 = pd.pivot_table(dfc, index = segmentacion, columns = 'Agrc5',
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas, fill_value = 0)

# td5.head()

In [31]:
td5.columns

MultiIndex([(  'Cantidad',     'sum',      'Farmacia'),
            (  'Cantidad',     'sum', 'No Considerar'),
            (  'Cantidad',     'sum',    'Otras vias'),
            (  'Cantidad',     'sum',     'Provision'),
            (   'Consumo',     'sum',      'Farmacia'),
            (   'Consumo',     'sum', 'No Considerar'),
            (   'Consumo',     'sum',    'Otras vias'),
            (   'Consumo',     'sum',     'Provision'),
            ('Prestacion', 'nunique',      'Farmacia'),
            ('Prestacion', 'nunique', 'No Considerar'),
            ('Prestacion', 'nunique',    'Otras vias'),
            ('Prestacion', 'nunique',     'Provision')],
           names=[None, None, 'Agrc5'])

In [32]:
# Nombres de columnas:
col_td5 = [
    
    "Cantidad Farmacia",
    "Cantidad No Considerar",
    "Cantidad Otras vias",
    "Cantidad Provision",
    
    "Consumo Farmacia",
    "Consumo No Considerar",
    "Consumo Otras vias",    
    "Consumo Provision",  

    "Prest. Farmacia",
    "Prest. No Considerar",
    "Prest. Otras vias",
    "Prest. Provision"]

# Renombro las columnas:
td5.columns = col_td5

# TD6 - Medicamentos por Origen

## Agr 6: Medicamentos por Cobertura

In [33]:
ruta_aux = 'C:\\modelos\\Auxiliares\\'
dfmed = pd.read_excel(ruta_aux + 'Aux_Prestaciones_especiales_med.xlsx')
dfmed.head()

,Prestacion,Tipo_med,Cobertura
0,ME00037909,R310,70
1,ME00053398,R310,70
2,ME00002232,R310,70
3,ME00044879,R310,70
4,ME00051142,R310,70


In [34]:
dfmed.Tipo_med.unique()

array(['R310', 'ME6000000 y ME1600085', 'MACBI'], dtype=object)

In [35]:
# Cruzamos con el dfc:
dfc = pd.merge( left = dfc, right = dfmed[["Prestacion","Tipo_med"]], 
               left_on='Prestacion', right_on='Prestacion', how = "left")

# Libero memoria:
dfmed = []

In [36]:
# Cálculos auxiliares para definir % cobertura:

# Creo el cálculo auxiliar con *10 para quedarnos con la decena de la cobertura
dfc['Cob_prec_sug'] = (dfc.Consumo / dfc.Precio_sug)*10

# Reemplazo valores +inf y -inf por 0
dfc['Cob_prec_sug'] = dfc['Cob_prec_sug'].replace([np.inf, -np.inf], 0)

# Redondeo a 0 y multiplico por 10 para quedarme con la decena
dfc['Cob_prec_sug'] = round(dfc['Cob_prec_sug'],0)*10

In [37]:
ruta_aux = 'C:\\modelos\\Auxiliares\\'
dfmed2 = pd.read_excel(ruta_aux + 'Aux_CobertMedicamentosPlan.xlsx')
dfmed2.head()

,Plan,Cobertura_med_id,Cobertura_desc,Marca_Manual
0,B1000,301,Cobertura 40%,NaN
1,B100J,301,Cobertura 40%,NaN
2,B1050,303,Cobertura 50%,NaN
3,B105J,303,Cobertura 50%,NaN
4,B1060,304,Cobertura 60%,x


In [38]:
# Cruzamos con el dfc:
dfc = pd.merge( left = dfc, right = dfmed2[["Plan","Cobertura_med_id"]], 
               left_on='Plan', right_on='Plan', how = "left")

# Libero memoria:
dfmed2 = []

In [39]:
# MARCA COBERTURA BASE

# Transformo las provisiones en función de su descripción
condiciones_c = [
    (dfc.Cobertura_med_id == 301) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 302) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 302) & (dfc.Cobertura_DW == 50) & (dfc.Cob_prec_sug == 50),    
    (dfc.Cobertura_med_id == 303) & (dfc.Cobertura_DW == 50) & (dfc.Cob_prec_sug == 50),
    (dfc.Cobertura_med_id == 304) & (dfc.Cobertura_DW == 60) & (dfc.Cob_prec_sug == 60),
    (dfc.Cobertura_med_id == 305) & (dfc.Cobertura_DW == 70) & (dfc.Cob_prec_sug == 70),
    (dfc.Cobertura_med_id == 306) & (dfc.Cobertura_DW == 80) & (dfc.Cob_prec_sug == 80),
    (dfc.Cobertura_med_id == 307) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 307) & (dfc.Cobertura_DW == 60) & (dfc.Cob_prec_sug == 60),
    (dfc.Cobertura_med_id == 308) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 310) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 312) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 312) & (dfc.Cobertura_DW == 60) & (dfc.Cob_prec_sug == 60),
    (dfc.Cobertura_med_id == 313) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 313) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 315) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40) 
]

# Lista de resultados en función de las selecciones
valores_condic_c = ['Cob. Base 40%', 
                  'Cob. Base 40%',
                  'Cob. Base 50%',
                  'Cob. Base 50%',
                  'Cob. Base 60%',
                  'Cob. Base 70%',
                  'Cob. Base 80%',
                  'Cob. Base 40%',
                  'Cob. Base 60%',
                  'Cob. Base 40%',
                  'Cob. Base 40%',
                  'Cob. Base 40%',
                  'Cob. Base 60%',
                  'Cob. Base 40%',
                  'Cob. Base 40%',
                  'Cob. Base 40%']     

# Generación de la columna calculada
dfc['Marca_Cob_Base'] = np.select(condiciones_c, valores_condic_c, default = 'No Cob. Base')

In [40]:
# MARCA COBERTURA 100%

condiciones = [(dfc.Cobertura_DW == 100) & (dfc.Cob_prec_sug == 100)]

# Lista de resultados en función de las selecciones
valores_condic = ['Cobertura 100%']  

# Generación de la columna calculada
dfc['Marca_Cobertura_100'] = np.select(condiciones, valores_condic, default = 'No Cobertura 100%')

In [41]:
# Genero la agrupación final med
condiciones = [
    # Tipo medicamento es MACBI, ME6000 o M1 o R310.
    dfc.Tipo_med == 'ME6000000 y ME1600085',
    dfc.Tipo_med.isnull() == 0,
    dfc.Marca_Cob_Base != 'No Cob. Base',
    dfc.Marca_Cobertura_100 == "Cobertura 100%",
    dfc["Tipo Orden ID"] == 4,
    dfc["Nomenclador ID"] == 'ME',
    dfc["Nomenclador ID"] == 'M1']
    
# Lista de resultados en función de las selecciones
valores_condic = ["Prest Genéricas", dfc.Tipo_med, dfc.Marca_Cob_Base, dfc.Marca_Cobertura_100, "Form. 4", "Otros", "Otros"]  

# Generación de la columna calculada
dfc['Marca_Cob_Final_Med'] = np.select(condiciones, valores_condic, default = 'No Considerar')

In [42]:
dfc['Marca_Cob_Final_Med'].unique()

array(['Form. 4', 'No Considerar', 'Prest Genéricas', 'Cob. Base 40%',
       'R310', 'Cobertura 100%', 'Cob. Base 60%', 'Cob. Base 50%',
       'Cob. Base 80%', 'Otros', 'MACBI'], dtype=object)

# TD7 - Marca PMO SanCor

## Agr 7: PMO SanCor

In [43]:
# Apertura para consumo: Agrupador consumos 7: se parte de la columna PMO

# Armado de la td:

td7 = pd.pivot_table(dfc, index = segmentacion, columns = 'Marca PMO',
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas, fill_value = 0)



td7.columns

MultiIndex([(  'Cantidad',     'sum', 'NO PMO'),
            (  'Cantidad',     'sum',    'PMO'),
            (   'Consumo',     'sum', 'NO PMO'),
            (   'Consumo',     'sum',    'PMO'),
            ('Prestacion', 'nunique', 'NO PMO'),
            ('Prestacion', 'nunique',    'PMO')],
           names=[None, None, 'Marca PMO'])

In [45]:
# Nombres de columnas:
col_td7 = [
    
    "Cantidad NO PMO",
    "Cantidad PMO",
    
    "Consumo NO PMO",
    "Consumo PMO",
 
    "Prest. NO PMO",
    "Prest. PMO"]

# Renombro las columnas:
td7.columns = col_td7

# Quiero armar la apertura de tipo internacion

In [46]:
dfc.columns

Index(['Periodo', 'Periodo_Prest', 'Persona', 'Fecha_nac', 'Edad Orden ID',
       'Sexo', 'Discapacidad', 'Posicion Asociado ID', 'Zona de Promocion ID',
       'Plan', 'Plan Agrupado', 'Origen Prestacion ID',
       'Origen Facturacion ID', 'Prestacion', 'Prestacion DESC',
       'Rubro Indicadores Consumo ID', 'Prestacion Acreedor ID',
       'Prestacion Acreedor DESC', 'Nomenclador ID', 'Nomenclador DESC',
       'Acreedor ID', 'Provision Acreedor ID', 'Posicion Acreedor ID',
       'Tipo Orden ID', 'Nro Orden ID', 'Tipo Orden Relacionada ID',
       'Orden Relacionada ID', 'Cobertura_DW', 'Tipo de Venta DESC',
       'Tipo de Venta ID', 'Subtipo de Venta DESC', 'Subtipo de Venta ID',
       'Segmento', 'Cantidad', 'Consumo', 'Precio_sug', 'Prest_sin_esp',
       'Prest sin espacios', 'Marca PMO', 'Agrc2', 'Agrc3', 'Agrc4', 'Agrc5',
       'Tipo_med', 'Cob_prec_sug', 'Cobertura_med_id', 'Marca_Cob_Base',
       'Marca_Cobertura_100', 'Marca_Cob_Final_Med'],
      dtype='object')

Procedimiento:

1- intentar poner el tipo internación en el reporte de consumos <br>
2- en caso de no poder realizar lo primero hacer una auxiliar con tipo-nro orden y tipo de internacion<br>
3- Separar por tipos de internación en las diferentes columnas<br>
4- Pensar qué vamos a hacer con las "internaciones no encontradas"<br>
5- Vamos a medir alguna "tasa de uso" o "alos" de las diferentes internaciones?.<br>

# Agrupamos las diferentes tablas

In [47]:
tdt = pd.concat([td1, td2, td3, td4, td5, td7], axis=1)
tdt.reset_index()
tdt.head()

Cantidad Total  Consumo Total  Prest. Dist. Total  \
Periodo_Prest Persona                                                      
202010        266                   8        1101.97                   6   
              452                   5        3366.78                   5   
              779                  22        4835.15                  19   
              780                  27        4422.58                   8   
              1219                 11        5360.73                  10   

                       Cantidad Amb.  Cantidad Int.  Cantidad Medic.  \
Periodo_Prest Persona                                                  
202010        266                  2              0                0   
              452                  1              0                0   
              779                 13              1                2   
              780                 21              0                0   
              1219                 4              0                0   

                       Cantidad NC.  Cantidad NA  Consumo Amb.  Consumo Int.  \
Periodo_Prest Persona                                                          
202010        266                 2            4       1158.04           0.0   
              452                 3            1       3298.20           0.0   
              779                 2            4       2685.60        1564.0   
              780                 2            4       5570.00           0.0   
              1219                3            4       7466.00           0.0   

                       ...  Prest. Farmacia  Prest. No Considerar  \
Periodo_Prest Persona  ...                                          
202010        266      ...                0                     6   
              452      ...                0                     5   
              779      ...                2                    17   
              780      ...                0                     8   
              1219     ...                0                    10   

                       Prest. Otras vias  Prest. Provision  Cantidad NO PMO  \
Periodo_Prest Persona                                                         
202010        266                      0                 0                1   
              452                      0                 0                2   
              779                      0                 0                3   
              780                      0                 0               11   
              1219                     0                 0                2   

                       Cantidad PMO  Consumo NO PMO  Consumo PMO  \
Periodo_Prest Persona                                              
202010        266                 7            1.30      1100.67   
              452                 3            1.30      3365.48   
              779                19         1589.25      3245.90   
              780                16         3151.30      1271.28   
              1219                9            1.30      5359.43   

                       Prest. NO PMO  Prest. PMO  
Periodo_Prest Persona                             
202010        266                  1           5  
              452                  2           3  
              779                  3          16  
              780                  2           6  
              1219                 2           8  

[5 rows x 81 columns]

# 4- Exportar la información a un .csv


Dentro del código queda pendiente:
1- Calcular rango de edad.
2- Cargar el diganóstico de todo el stock para levantar una vez.
3- Definir una función para cambio de nombres.
4- Definir una función para carga de dataframes desde excel.
5- Definir todos los parámetros que fueron necesarios.



In [48]:
# Genero un dataframe con la td
base_consumos = pd.DataFrame(tdt)

# Exporto la información en csv:
base_consumos.to_csv(archivo_salida, encoding = 'latin1', decimal=',')
# A modo de ejemplo:
#  df.to_csv('./exportaciones/' + supermercado + '-' + str(fecha_exportado) + '.csv', encoding = 'latin1')